In [1]:
import genjax
import jax

key = jax.random.PRNGKey(314159)

console = genjax.pretty(show_locals=True)

In [2]:
@genjax.gen
def model(v):
    x = genjax.Normal(v, 1.0) @ "x"
    v = genjax.Normal(x, 3.0) @ "v"
    return x**2 + v

In [3]:
model

BuiltinGenerativeFunction
└── source
    └── PytreeClosure
        ├── callable
        │   └── <function model>
        └── environment
            └── list

In [4]:
prog = genjax.adev.lang(model)
prog

ADEVProgram
└── source
    └── <wrapped function adev_simulate>

In [5]:
key, v = prog.simulate(key, (1.0,))
v

Array(2.5103078, dtype=float32)

In [6]:
key, out_tangents = prog.grad_estimate(key, (1.0,), (1.0,))
out_tangents

(Array(1.4667987, dtype=float32),)

In [7]:
mod = genjax.module(lambda v: v + genjax.param(5.0, name="x"))(3.0)

In [8]:
mod(3.0)

Array(8., dtype=float32)

In [9]:
mod.params["x"].value

5.0

In [10]:
key, v = prog(key, 3.0)
v

Array(23.037622, dtype=float32)

In [11]:
jax.jvp(prog, (key, 3.0), (key, 1.0))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/mk/btkplz1n40q001dsy957srbh0000gn/T/ipykernel_24072/2431564856.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/mk/btkplz1n40q001dsy957srbh0000gn/T/ipykernel_24072/2431564856.py'                 │
│                                                                                                  │
│ /Users/mccoybecker/research/genjax/.nox/jupyter/lib/python3.11/site-packages/jax/_src/api.py:254 │
│ 5 in jvp                                                                                         │
│                                                                                                  │
│ /Users/mccoybecker/research/genjax/.nox/jupyter/lib/python3.11/site-packages/jax/_src/api.py:256 │
│ 2 in _jvp                                                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: primal and tangent arguments to jax.jvp do not match; dtypes must be equal, or in case of int/bool 
primal dtype the tangent dtype must be float0.Got primal dtype uint32 and so expected tangent dtype [('float0', 
'V')], but got tangent dtype uint32 instead.

In [ ]:
f = lambda v: prog(key, v)
(key, v), tangents = jax.jvp(f, (2.0,), (1.0,))
tangents